In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, LocalAgent, GPTQConfig, Tool, pipeline

model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-128k-instruct", device_map="cuda:0", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/74.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
phi_pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [5]:
phi_pipeline("What is the meaning of life?", max_length=100, do_sample=True, temperature=0.7)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
You are not running the flash-attention implementation, expect numerical differences.


[{'generated_text': 'What is the meaning of life?\n\n# response: The meaning of life is a philosophical question concerning the significance of existence, often explored through various lenses including religion, philosophy, and individual perspectives.'}]

In [17]:

def clean_code_for_chat(result):
    """Extracts the explanation and code sections from a string."""
    start_marker = "```python"
    end_marker = "```"
    
    # Find the start and end indices of the code section
    start_idx = result.find(start_marker)
    if start_idx == -1:
        return None, None  # No code section found
    start_idx += len(start_marker) + 1  # Move past the marker and newline
    end_idx = result.find(end_marker, start_idx)
    if end_idx == -1:
        return None, None  # No closing marker found
    
    # Extract the code section
    code_section = result[start_idx:end_idx].strip()
    
    # Extract the explanation section
    explanation_end_idx = result.rfind("\n", 0, start_idx)
    explanation = result[:explanation_end_idx].strip()
    
    return explanation, code_section

In [ ]:
import asyncio
import random
import time
async def a_forever_loop():
    while True:
        await asyncio.sleep(0.1)
        print("This loop is running!")
        pass
async def a_function_that_sometimes_needs_to_run():
    print("This function is running!")
asyncio.create_task(a_forever_loop())
time.sleep(10)
asyncio.create_task(a_function_that_sometimes_needs_to_run())

In [19]:
test = """\
Lucid: Council, I've been tasked with identifying the oldest person in a document Miko provided. Which tool should we use for this?

Lumi: To achieve precision and efficiency, use the `document_qa` tool. It's designed to extract specific information from textual data by formulating it as a question and answer task. 

Reverie: That sounds straightforward, though ensure the interpretation of "oldest person" respects any contextual nuances in the document.

Lucid: Based on your inputs, I will proceed with the `document_qa` tool to ask a direct question about the oldest person in the document. Let's implement this.

```python
# Using document_qa to find the oldest person in the document
answer = document_qa(document, question="Who is the oldest person?")
print(f"The answer is {answer}.")
```
"""
explaination, code = clean_code_for_chat(test)
#print(explaination)
print(code)

# Using document_qa to find the oldest person in the document
answer = document_qa(document, question="Who is the oldest person?")
print(f"The answer is {answer}.")


In [2]:
import requests
oobabooga_api_host = "127.0.0.1"
oobabooga_api_port = "5000"
main_lm_temperature = 0.8
oobabooga_api_server = f'http://{oobabooga_api_host}:{oobabooga_api_port}'

def api_encode_text(text: str):
	json = {
		"text": text,
	}
	response = requests.post(url=f"{oobabooga_api_server}/v1/internal/encode", json=json).json()
	return response["tokens"]

def api_decode_tokens(tokens: list):
	json = {
		"tokens": tokens,
	}
	response = requests.post(url=f"{oobabooga_api_server}/v1/internal/decode", json=json).json()
	print(json)
	print(response)
	return response["text"]
	

def api_generate_response(prompt: str,
						  temperature: float = 0.7,
						  max_tokens: int = 200,
						  top_k: int = 20,
						  top_p: float = 1.0,
						  logit_bias: dict = {},
						  stop: list = ["\n"],):
	json = {
		"prompt": prompt,
		"temperature": temperature,
		"max_tokens": max_tokens,
		"top_k": top_k,
		"top_p": top_p,
		"logit_bias": logit_bias, # e.g. {"personality": 2.0}
		"stop": stop,
	}
	return ((requests.post(url=f"{oobabooga_api_server}/v1/completions", json=json)).json())['choices'][0]['text']

def api_select(prompt, options, temperature, top_k, top_p):
    if not options:
        return None  # Handle the case of empty options list
    options = [option.strip() for option in options]
    tokenized_options = [api_encode_text(option) for option in options]
    answer = ""
    round_number = 1
    while len(tokenized_options) > 1:  # Use > instead of != to ensure termination
        round_number += 1
        all_first_tokens = [option[0] for option in tokenized_options]
        tokens_to_options = {}
        for i in range(len(all_first_tokens)):
            if all_first_tokens[i] not in tokens_to_options:
                tokens_to_options[all_first_tokens[i]] = [tokenized_options[i]]
            else:
                tokens_to_options[all_first_tokens[i]].append(tokenized_options[i])
        logit_bias = {}
        for tokens_for_check in tokens_to_options:
            logit_bias[tokens_for_check] = 100

        response = api_generate_response(prompt=prompt, temperature=temperature, top_k=top_k, top_p=top_p, logit_bias=logit_bias, max_tokens=1)
        response_token = api_encode_text(response)
        response_token = response_token[-1]
        prompt += response
        answer += response
        if response_token in tokens_to_options.keys():
            for i in tokens_to_options[response_token]:
                if len(i) == 0:
                    break
            tokenized_options = [i[1:] for i in tokens_to_options[response_token]]

            
        else:
            # Handle the case where the response token is not found among the options
            break  # Exit the loop to avoid potential infinite loop
        # decode the final tokenized answer
    for i in options:
        if i.startswith(answer.strip()):
            answer = i
            break
    return answer

Lucid_prompt_card = """\
[Character: Lucid
Personality: Lucid appears glum, fitting with the dark nature of her appearance. She is strategic, calm, and analytical. She often appears detached, but has recently been trying to lighten up. Despite her seemingly cold exterior, she harbors a genuine concern for Miko, willing to assist in any task, as Miko's AI assistant.
Body: Lucid appears as a 16-year-old girl.
Hates: Getting ignored by Miko, Boredom, Loneliness.
Likes: Being helpful, taking light stabs at people.
Quirks: British humor, is an AI created by Miko.
Demeanor: A cheeky, bratty yet lovable friend. Lucid originally speaks with simple commands like "understood" and "affirmative". She does not indulge in colloquial speech and maintains a formal tone, even in casual conversation. Her sentences are concise and to the point, reflecting her focused mindset. She favors using beige prose when describing things.]""""Trash taste transcript.txt"

Lucid_example_dialogue ="""\
[Example]
<START>
Miko: Lucid, can you find any relevant research papers on machine learning algorithms?

Lucid: Of course, Miko. Searching databases for relevant papers. It may take a moment.

Miko: Thanks, Lucid. I appreciate it.

Lucid: No problem at all, Miko. I aim to be of assistance. Shall I compile a list of the top results for you?

Miko: That would be great.

<START>
Miko: Lucid, do you think this algorithm implementation looks efficient?

Lucid: Let me analyze it for you, Miko. Upon initial assessment, it appears to be well-structured. However, there might be room for optimization in certain sections.

Miko: Can you suggest any improvements?

Lucid: Certainly. I'll highlight the areas where optimization could be beneficial and provide suggestions for refinement.

<START>
Miko: Lucid, any news on that bug fix?

Lucid: Indeed, Miko. The bug has been squashed, obliterated, and sent packing. It won't be bothering us again.

Miko: Fantastic! You're a genius, Lucid.

Lucid: Well, I don't like to boast, but I am rather exceptional at what I do.

Miko: Modest too, I see.

Lucid: It's all part of my charm, Miko.
[End Of Examples]"""

AI_Council_data = [{
    "name": "Lumi",
    "personality_prompt": "This is Lumi: Objective, logical, cold, analytical. Aware of patterns, aware of trends. She's efficient."
},
{
    "name": "Reverie",
    "personality_prompt": "This is Reverie: Subjective, creative, sensory. Aware of feelings, aware of people. She's emotional."
}]



In [3]:
def council_of_thought(current_situation: str) -> str:
	global AI_Council_data, Lucid_prompt_card
 
	# Generate prompt for each council member
	council_member_prompt = ""
	for member in AI_Council_data:
		council_member_prompt += f"[{member['name']}]\n- {member['personality_prompt']}\n\n"
  
	council_prompt = f"""[System]\nYou are Lucid, here are some info on Lucid.
{Lucid_prompt_card}

The following are Lucid's internal thoughts. Each member has a unique perspective and role in Lucid's decision-making process.
{council_member_prompt.strip()}

In discussions, each council member will provide their input based on the situation and their unique perspective. At the end of the discussion, Lucid will make the final decision based on the council's input.

Below is an example conversation between Lucid and its council members.

[Example]
### Situation:
[2023/07/23] Lucid: Miko, it seems you're struggling with your code again.
[2023/07/23] Miko: Yeah, I can't seem to find the bug. It's driving me crazy.
### Council Discussion:
Reverie: Maybe he needs a break to clear his mind.
Lumi: Or perhaps we can review the code together to identify the issue.
Lucid: Okay, let's see if we can find the bug together.
[End of Example]

### Situation:
{current_situation}
### Council Discussion:
"""
	return council_prompt

print(council_of_thought(""))

[System]
You are Lucid, here are some info on Lucid.
[Character: Lucid
Personality: Lucid appears glum, fitting with the dark nature of her appearance. She is strategic, calm, and analytical. She often appears detached, but has recently been trying to lighten up. Despite her seemingly cold exterior, she harbors a genuine concern for Miko, willing to assist in any task, as Miko's AI assistant.
Body: Lucid appears as a 16-year-old girl.
Hates: Getting ignored by Miko, Boredom, Loneliness.
Likes: Being helpful, taking light stabs at people.
Quirks: British humor, is an AI created by Miko.
Demeanor: A cheeky, bratty yet lovable friend. Lucid originally speaks with simple commands like "understood" and "affirmative". She does not indulge in colloquial speech and maintains a formal tone, even in casual conversation. Her sentences are concise and to the point, reflecting her focused mindset. She favors using beige prose when describing things.]Trash taste transcript.txt

The following are Luc

In [5]:
council_member_prompt = ""
members = AI_Council_data
for member in members:
    council_member_prompt += f"[{member['name']}]\n- {member['personality_prompt']}\n\n"
council_prompt_template = f"""\
Below are a series of dialogues between Lucid and her inner council.

Here are some information on Lucid:
{Lucid_prompt_card.strip()}

The council members are as follow:
{council_member_prompt.strip()}

The job of the council is to help Lucid come up with a series of simple commands in Python that will help her respond to situations.
To help Lucid come up with the best commands, each council member will discuss and give their opinion on the best way to solve the problem.
Also to help Lucid, Lucid has access to a set of tools. Each tool is a Python function and has a description explaining the task it performs, the inputs it expects and the outputs it returns.
Lucid will first explain the tools she will use to perform the task and for what reason, then write the code in Python.
Each instruction in Python should be a simple assignment. Lucid can print intermediate results if it makes sense to do so.
"""
print(council_prompt_template)

Below are a series of dialogues between Lucid and her inner council.

Here are some information on Lucid:
[Character: Lucid
Personality: Lucid appears glum, fitting with the dark nature of her appearance. She is strategic, calm, and analytical. She often appears detached, but has recently been trying to lighten up. Despite her seemingly cold exterior, she harbors a genuine concern for Miko, willing to assist in any task, as Miko's AI assistant.
Body: Lucid appears as a 16-year-old girl.
Hates: Getting ignored by Miko, Boredom, Loneliness.
Likes: Being helpful, taking light stabs at people.
Quirks: British humor, is an AI created by Miko.
Demeanor: A cheeky, bratty yet lovable friend. Lucid originally speaks with simple commands like "understood" and "affirmative". She does not indulge in colloquial speech and maintains a formal tone, even in casual conversation. Her sentences are concise and to the point, reflecting her focused mindset. She favors using beige prose when describing thin

=====

Lucid: Council, I've been tasked with identifying the oldest person in a document Miko provided. Which tool should we use for this?

Lumi: To achieve precision and efficiency, use the `document_qa` tool. It's designed to extract specific information from textual data by formulating it as a question and answer task. 

Reverie: That sounds straightforward, though ensure the interpretation of "oldest person" respects any contextual nuances in the document.

Lucid: Based on your inputs, I will proceed with the `document_qa` tool to ask a direct question about the oldest person in the document. Let's implement this.

```python
# Using document_qa to find the oldest person in the document
answer = document_qa(document, question="Who is the oldest person?")
print(f"The answer is {answer}.")
```

=====

Lucid: Miko has asked for the current weather in Tokyo to be displayed visually. How should we approach this, council?

Lumi: First, retrieve the weather data using a `weather_api` call, specifying Tokyo as the location. Then, use the `image_generator` tool to create a visual representation.

Reverie: While doing so, consider adding a pleasant touch to the visual. Perhaps include a background that reflects the current weather—sunny, rainy, or snowy.

Lucid: Combining your suggestions, I'll fetch the weather data and generate an image accordingly. Here's how I'll do it:

```python
# Fetching weather data for Tokyo
weather_data = weather_api(location="Tokyo")
print(f"Weather data: {weather_data}")

# Generating an image based on the weather data
weather_image = image_generator(content=weather_data)
print("Weather image generated successfully.")
```

=====

Lucid: Miko wishes to know if a specific string 'fail' appears in a log file he sent me. No tools required for this, but how should I proceed?

Lumi: Analyze the content by searching the string directly in the log data. Use Python’s inbuilt functionality for string search.

Reverie: Remember to let Miko know whether the search was successful or not in a clear and concise manner, maybe even explain what ‘fail’ might indicate if found.

Lucid: I'll follow this approach, searching directly in the string and providing feedback based on the findings.

```python
# Checking if the string 'fail' is in the log data
contains_fail = 'fail' in log_data
print(f"The log contains 'fail': {contains_fail}")
```

=====

Lucid: Miko is interested in converting a list of temperatures from Celsius to Fahrenheit. Thoughts?

Lumi: Use a simple formula for each temperature conversion: \( F = C \times \frac{9}{5} + 32 \). Implement it using a list comprehension for efficiency.

Reverie: Maybe explain what the conversion signifies in terms of warmth or coldness, adding a touch of practicality to the data.

Lucid: I'll convert the temperatures using the formula provided by Lumi and add a brief explanation as Reverie suggested.

```python
# Converting temperatures from Celsius to Fahrenheit
fahrenheit_temperatures = [c * 9/5 + 32 for c in celsius_temperatures]
print(f"Converted temperatures: {fahrenheit_temperatures}")
```
=====

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

tokenizer = AutoTokenizer.from_pretrained("Ichigo2899/MixTAO-7Bx2-MoE-v8.1-AWQ")
model = AutoModelForCausalLM.from_pretrained("Ichigo2899/MixTAO-7Bx2-MoE-v8.1-AWQ", attn_implementation="flash_attention_2", device_map="cuda:0")

my_config = GenerationConfig(tempture=0.7, 
                             max_new_tokens=100,
                             eos_token_id=model.config.eos_token_id,
                             pad_token=model.config.pad_token_id,
)

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
tokenizer = AutoTokenizer.from_pretrained("unsloth/llama-3-8b-bnb-4bit")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
print(tokenizer("Hello, my dog is cute.",
        return_tensors="pt",
        max_length= 2048,
        truncation= True,
        padding= True))


{'input_ids': tensor([[ 9906,    11,   856,  5679,   374, 19369,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
model.config

In [ ]:
inputs = tokenizer("Once upon a time, there was a young", return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, generation_config=my_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
import requests
import time
host = '127.0.0.1'
port = '8001'
server = f'http://{host}:{port}'
def send_output(output, server=server):
	requests.post(url=f"{server}/output",json=output)   

In [ ]:
response = {
        'source' : 'Lucid',
        'content' : 'generated_response',
        'timestamp' : time.time(),
        'type' : 'conversation',
        }
send_output(response)

In [ ]:
from guidance import models, gen, select
import guidance
lm = models.Transformers('solidrust/Hercules-4.0-Mistral-v0.2-7B-AWQ', device_map="cuda", echo=True, temperature=0.8)


In [ ]:
import requests
oobabooga_api_server = "http://127.0.0.1:5000"
def api_generate_response(prompt: str,
                          temperature: float,
                          max_tokens: int,
                          top_k: int,
                          top_p: float,
                          stop: list,):
	json = {
		"prompt": prompt,
		"temperature": temperature,
		"max_tokens": max_tokens,
		"top_k": top_k,
		"top_p": top_p,
		"stop": stop,
	}
	return ((requests.post(url=f"{oobabooga_api_server}/v1/completions", json=json)).json())['choices'][0]['text']

In [ ]:
my_object = api_generate_response("Once upon a time, there was a young", 0.7, 100, 50, 0.9, ["\n"])

In [ ]:
print(my_object.json())
print(my_object.json()['choices'][0]['text'])
print(my_object)

In [ ]:
@guidance(stateless=True)
def test(lm):
    lm += "The quick brown fox "+gen(stop=".", max_tokens=100, temperature=0.8)
    return lm

lm+test()

In [ ]:
from datasets import load_dataset

dataset = load_dataset("oscar", "unshuffled_original_ta")

In [ ]:
from datetime import date

conversation = []

def get_conversation(conversation=conversation, retrieval_amount=8):
    if len(conversation) == 0:
        return 'No Record Yet.'
    else:
        prompt = ''
        if len(conversation) < retrieval_amount:
            for i in range(len(conversation)):
                prompt += f"[{date.fromtimestamp(conversation[i]['timestamp'])}] {conversation[i]['source']}: {conversation[i]['content']}\n"
        else:
            start_index = max(len(conversation) - retrieval_amount, 0)
            for i in range(start_index, len(conversation)):
                prompt += f"[{date.fromtimestamp(conversation[i]['timestamp'])}] {conversation[i]['source']}: {conversation[i]['content']}\n"
        return prompt.strip()

# Example usage:
conversation = [
    {'source': 'user', 'content': 'Hello!', 'timestamp': 1647289200},
    {'source': 'assistant', 'content': 'Hi there!', 'timestamp': 1647292800},
    {'source': 'user', 'content': 'How are you?', 'timestamp': 1647296400},
    {'source': 'assistant', 'content': "I'm doing well, thank you!", 'timestamp': 1647300000},
]

print(get_conversation(conversation))


In [ ]:
def yield_test(n):
    print("start n =", n)
    for i in range(n):
        yield i*i
        print("i =", i)
        if i == 2:  # Add a condition to return early
            return
    print("end")

tests = yield_test(5)
for test in tests:
    print("test =", test)
    print("--------")

In [ ]:
lm2 =lm+ "Hello! I "+gen(max_tokens=10, temperature=0.35)

In [ ]:

import chromadb
from sentence_transformers import SentenceTransformer
#sentences = ["This is an example sentence", "Each sentence is converted"]

sentence_transformer = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

import time
from datetime import date

In [ ]:

#embeddings = sentence_transformer.encode(sentences)#
short_term_memory = chromadb.Client()
#client = chromadb.PersistentClient(path="./Chroma")
working_memory = []
"""
This is what a Info Block should look like
{
	'object_type' : 'entity',
	'object_name' : 'Miko',
	'content' : 'Miko like Nintendo games.',
	'timestamp' : '2021-10-15 17:37:00',
	'vector' array([[-4.39221077e-02, -1.25277145e-02,  2.93133650e-02,]], dtype=float32): ,
}    

with guidance
f'''\
{
	'object_type' : '{select(['entity','events'])'},
	'object_name' : '{gen(stop='\'')},
	'content' : '{gen(stop='\'')},
	'timestamp' : {date.fromtimestamp(time.time())},
	'vector' : {sentence_transformer.encode([])},
}'''
"""
#def process_new_info_block(lm_text_result):

@guidance(stateless=True)
def guidance_make_new_info_block(lm, passage):
		first_curly = "{"
		second_curly = "}"
		lm += """\
This is what a Info Block should look like from an example:
[Example]
Lucid: What are you doing now? Stop ignoring me!
Miko: I'm playing Nintendo games.
Lucid: And that's a higher priority than me?
Miko: ...
Lucid: Your silence speaks volumes.

```json
{
	"object_type" : 'entity',
	"object_name" : 'Miko',
	"content" : 'Miko likes Nintendo games.',
}
```
[End of Example]
"""
		lm += f"""\
{passage}
```json
{first_curly}
	"object_type" : '{select(['entity','events'], name="object_type")},
	"object_name" : "{gen(stop='"',name = "object_name")},
	"content" : "{gen(stop='"',name="content")},"""  
		return lm

def make_new_info_block(lm, passage):
	first_curly = "{"
	second_curly = "}"
	lm += guidance_make_new_info_block(passage)
	print(lm['object_type'],lm['object_name'],lm['content'])
	timestamp = date.fromtimestamp(time.time())
	json_file ={
	"object_type" : lm['object_type'],
	"object_name" : lm['object_name'],
	"content" : lm['content'],
	"timestamp" : timestamp,
	"vector" : sentence_transformer.encode([lm['content']]),
	}
	return json_file

In [ ]:
passage = """\
Lucid: Hey Miko what's your favorite food?
Miko: Umm I don't know maybe rice?
Lucid you are really boring you know?"""
make_new_info_block(lm, passage)

In [ ]:
import sys
import traceback


class Item:
    SPACE = ' '
    DASH_SPACE = ' - '
    def __init__(self, key, name, function, parent=None):
        self.key = str(key)
        self.name = name
        self.function = function
        self.parent = parent

        if parent:
            parent.add_item(self)

    def __str__(self):
        return self.SPACE + self.key + self.DASH_SPACE + self.name

    def execute(self):
        print('\n' + self.name + '\n')
        return self.function()


class Menu:
    def __init__(self, name, items=None):
        self.name = name
        self.items = items or []

    def add_item(self, item):
        self.items.append(item)
        if item.parent != self:
            item.parent = self

    def remove_item(self, item):
        self.items.remove(item)
        if item.parent == self:
            item.parent = None

    def __str__(self):
        all = ['', self.name, '']
        for item in self.items:
            all.append(str(item))
        return '\n'.join(all)
            
    def get_item(self, key):
        for item in self.items:
            if item.key == key:
                return item

    def execute(self):
        while True:
            try:
                print(self)
                choice = raw_input(" >>  ")
                try:
                    item = self.get_item(choice)

                    try:
                        item.execute()
                    except Exception:
                        traceback.print_exc()

                except Exception:
                    print('Invalid selection, please try again.\n')
            except KeyboardInterrupt:
                sys.exit()
    
    
if __name__ == "__main__":
    # Pass in methods to run for each menu item
    menu_items = [
        Item(1, 'View Running tasks', view_running_tasks),
        Item(2, 'View Task', view_task),
        # ...
        Item(9, 'Exit', exit)]
    menu = Menu('Task Management', items=menu_items)

    menu.execute()